In [1]:
import pandas as pd

nodes = pd.read_csv("jhotdraw-comm_louvain-nodes.csv")
edges = pd.read_csv("jhotdraw-comm_louvain-edges.csv")

nodes.shape, edges.shape

((34, 13), (33, 5))

note: communities [hidef] are from cytoscape's community detection with default parameters:

- algorithm = HiDeF
- weight column = none
- parameters:
  - max resolution = 50
  - consensus threshold = 75
  - persistent threshold = 5
  - algorithm = louvain
  - target community number = (blank)

communities [louvain] uses the parameters:

- algorithm = Louvain
- weight column = none
- parameters:
  - directed = True
  - overlapping = False
  - random gen seed = 42
  - resolution = 5
  - deep hierarchy = False

In [2]:
nodes['members'] = [set(l.split(" ")) for l in nodes['CD_MemberList']]

nodes.sort_values(by=['level'], ascending=False, inplace=True)
indices = list(nodes.index)

df = nodes[['name', 'CD_MemberList_Size', 'members', 'level']]
pd.concat([df.head(5), df.tail(5)])

,name,CD_MemberList_Size,members,level
17,C32293,4,"{EllipseFigure, ColorMap, FigureAttributes, At...",1
25,C32301,3,"{ConnectionHandle, ConnectionFigure, Connectio...",1
19,C32295,4,"{ChangeConnectionStartHandle, Connector, Chang...",1
20,C32296,4,"{PolygonScaleHandle, PolygonFigure, PolygonHan...",1
21,C32297,4,"{FontSizeHandle, NodeFigure, NumberTextFigure,...",1
5,C32281,7,"{ActionTool, DragTracker, SelectAreaTracker, F...",1
4,C32280,7,"{BouncingDrawing, AnimationDecorator, Standard...",1
3,C32279,7,"{CutCommand, DeleteCommand, FigureTransferComm...",1
2,C32278,8,"{GroupCommand, ChangeAttributeCommand, AlignCo...",1
0,C32312,152,"{PaletteIcon, PolyLineLocator, ReverseFigureEn...",0


note: level is currently manually edited in cytoscape. should be automated:

- visited nodes <- []
- remaining nodes <- all nodes
- find sink (node with no outgoing edges)
- assign level 0 to sink
- add sink to visited nodes
- remove sink from remaining nodes
- while there are remaining nodes:
  - find nodes in remaining nodes that has edge to visited nodes
  - assign level +1 to those nodes
  - add those nodes to visited nodes
  - remove those nodes from remaining nodes

In [3]:
nodes['my_members_str'] = ["" for i in indices]
df = nodes[['name', 'CD_MemberList_Size', 'members', 'my_members_str']]
pd.concat([df.head(5), df.tail(5)])

,name,CD_MemberList_Size,members,my_members_str
17,C32293,4,"{EllipseFigure, ColorMap, FigureAttributes, At...",
25,C32301,3,"{ConnectionHandle, ConnectionFigure, Connectio...",
19,C32295,4,"{ChangeConnectionStartHandle, Connector, Chang...",
20,C32296,4,"{PolygonScaleHandle, PolygonFigure, PolygonHan...",
21,C32297,4,"{FontSizeHandle, NodeFigure, NumberTextFigure,...",
5,C32281,7,"{ActionTool, DragTracker, SelectAreaTracker, F...",
4,C32280,7,"{BouncingDrawing, AnimationDecorator, Standard...",
3,C32279,7,"{CutCommand, DeleteCommand, FigureTransferComm...",
2,C32278,8,"{GroupCommand, ChangeAttributeCommand, AlignCo...",
0,C32312,152,"{PaletteIcon, PolyLineLocator, ReverseFigureEn...",


In [5]:
visited = { n for n in nodes.loc[indices[0],'members'] }
nodes.loc[indices[0],'my_members_str'] = ' '.join(visited)


In [6]:
for idx in range(1,len(indices)):
  i = indices[idx-1]
  j = indices[idx]
  my_nodes = nodes.loc[j,'members'] - visited
  visited |= my_nodes
  # print(nodes.loc[j,'name'])
  # print("then",nodes.loc[j,'members'])
  # print(" now",my_nodes)
  nodes.loc[j,'my_members_str'] = ' '.join(my_nodes)

nodes['my_members'] = [set(l.split(" ")) if l else set() for l in nodes['my_members_str']]
nodes['my_members_size'] = [len(l) for l in nodes['my_members']]


In [7]:
df = nodes[['name', 'CD_MemberList_Size', 'members', 'my_members_size', 'my_members', 'level']]

pd.concat([df.head(5), df.tail(5)])

,name,CD_MemberList_Size,members,my_members_size,my_members,level
17,C32293,4,"{EllipseFigure, ColorMap, FigureAttributes, At...",4,"{EllipseFigure, ColorMap, FigureAttributes, At...",1
25,C32301,3,"{ConnectionHandle, ConnectionFigure, Connectio...",3,"{ConnectionHandle, ConnectionFigure, Connectio...",1
19,C32295,4,"{ChangeConnectionStartHandle, Connector, Chang...",4,"{ChangeConnectionStartHandle, Connector, Chang...",1
20,C32296,4,"{PolygonScaleHandle, PolygonFigure, PolygonHan...",4,"{PolygonScaleHandle, PolygonFigure, PolygonHan...",1
21,C32297,4,"{FontSizeHandle, NodeFigure, NumberTextFigure,...",4,"{FontSizeHandle, NodeFigure, NumberTextFigure,...",1
5,C32281,7,"{ActionTool, DragTracker, SelectAreaTracker, F...",7,"{ActionTool, DragTracker, SelectAreaTracker, F...",1
4,C32280,7,"{BouncingDrawing, AnimationDecorator, Standard...",7,"{BouncingDrawing, AnimationDecorator, Standard...",1
3,C32279,7,"{CutCommand, DeleteCommand, FigureTransferComm...",7,"{CutCommand, DeleteCommand, FigureTransferComm...",1
2,C32278,8,"{GroupCommand, ChangeAttributeCommand, AlignCo...",8,"{GroupCommand, ChangeAttributeCommand, AlignCo...",1
0,C32312,152,"{PaletteIcon, PolyLineLocator, ReverseFigureEn...",0,{},0


In [8]:
from functools import reduce

all = reduce(lambda x,y: x|y, nodes['my_members'])
len(all)

152

In [10]:
nodes.columns

Index(['CD_AnnotatedMembers', 'CD_AnnotatedMembers_Overlap',
       'CD_AnnotatedMembers_Pvalue', 'CD_AnnotatedMembers_Size',
       'CD_CommunityName', 'CD_Labeled', 'CD_MemberList',
       'CD_MemberList_LogSize', 'CD_MemberList_Size', 'level', 'name',
       'selected', 'shared name', 'members', 'my_members_str', 'my_members',
       'my_members_size'],
      dtype='object')

In [11]:
c_nodes = nodes.drop(nodes[nodes.my_members_str.map(len)==0].index)
c_nodes = c_nodes.drop(columns=[
      'CD_AnnotatedMembers', 'CD_AnnotatedMembers_Overlap',
      'CD_AnnotatedMembers_Pvalue', 'CD_AnnotatedMembers_Size',
      'CD_CommunityName', 'CD_Labeled', 'CD_MemberList',
      'CD_MemberList_LogSize', 'CD_MemberList_Size',
      'selected', 'shared name', 'members', 'my_members_str', 
      'my_members_size'])
c_nodes.rename(columns={'name':'id', 'my_members':'members'}, inplace=True)
df = c_nodes
pd.concat([df.head(5), df.tail(5)])

,level,id,members
17,1,C32293,"{EllipseFigure, ColorMap, FigureAttributes, At..."
25,1,C32301,"{ConnectionHandle, ConnectionFigure, Connectio..."
19,1,C32295,"{ChangeConnectionStartHandle, Connector, Chang..."
20,1,C32296,"{PolygonScaleHandle, PolygonFigure, PolygonHan..."
21,1,C32297,"{FontSizeHandle, NodeFigure, NumberTextFigure,..."
6,1,C32282,"{PolyLineFigure, PolyLineHandle, ArrowTip, Pol..."
5,1,C32281,"{ActionTool, DragTracker, SelectAreaTracker, F..."
4,1,C32280,"{BouncingDrawing, AnimationDecorator, Standard..."
3,1,C32279,"{CutCommand, DeleteCommand, FigureTransferComm..."
2,1,C32278,"{GroupCommand, ChangeAttributeCommand, AlignCo..."


In [12]:
for line in [' '.join(members) for members in c_nodes['members']]:
  print(':', line)

: EllipseFigure ColorMap FigureAttributes AttributeFigure
: ConnectionHandle ConnectionFigure ConnectionTool
: ChangeConnectionStartHandle Connector ChangeConnectionHandle ChangeConnectionEndHandle
: PolygonScaleHandle PolygonFigure PolygonHandle AbstractHandle
: FontSizeHandle NodeFigure NumberTextFigure TextFigure
: FigureEnumeration FigureEnumerator GroupFigure BringToFrontCommand
: DrawingChangeEvent DrawingChangeListener Drawing
: AbstractFigure FigureChangeListener FigureChangeEventMulticaster
: ElbowConnection ElbowHandle LineConnection
: SouthEastHandle WestHandle SouthWestHandle NorthEastHandle NorthWestHandle NorthHandle EastHandle RelativeLocator SouthHandle BoxHandleKit LocatorHandle
: Animatable JavaDrawApplet Animator
: AbstractConnector ShortestDistanceConnector LocatorConnector
: ReverseVectorEnumerator ReverseFigureEnumerator SendToBackCommand
: DrawingEditor DrawingView
: PertDependency PertFigure
: FloatingTextField URLTool
: SelectionTool HandleTracker Handle MySele

note: community nodes were named individually by examining its members. some ideas for automation:

- use the name of the largest class
- use the name of "central" class in the community (e.g., most ingoing + outgoing edges?)
- find most recurring word from class names?

In [13]:
with open("comm_louvain.txt", "r") as file:
	names = [line.split(':')[0] for line in file.readlines()]
len(names)==len(set(names))

True

In [14]:
c_nodes['name'] = names
c_nodes['id'] = names
c_nodes['members_str'] = [' '.join(m) for m in c_nodes['members']]
# for line in [' '.join(s) for s in c_nodes['members']]:
#   print(line)
df = c_nodes

pd.concat([df.head(5), df.tail(5)])

,level,id,members,name,members_str
17,1,FIGURE_ATTRIBUTE,"{EllipseFigure, ColorMap, FigureAttributes, At...",FIGURE_ATTRIBUTE,EllipseFigure ColorMap FigureAttributes Attrib...
25,1,CONNECTION,"{ConnectionHandle, ConnectionFigure, Connectio...",CONNECTION,ConnectionHandle ConnectionFigure ConnectionTool
19,1,CONNECTION_CHANGE,"{ChangeConnectionStartHandle, Connector, Chang...",CONNECTION_CHANGE,ChangeConnectionStartHandle Connector ChangeCo...
20,1,POLYGON,"{PolygonScaleHandle, PolygonFigure, PolygonHan...",POLYGON,PolygonScaleHandle PolygonFigure PolygonHandle...
21,1,TEXT,"{FontSizeHandle, NodeFigure, NumberTextFigure,...",TEXT,FontSizeHandle NodeFigure NumberTextFigure Tex...
6,1,LINE,"{PolyLineFigure, PolyLineHandle, ArrowTip, Pol...",LINE,PolyLineFigure PolyLineHandle ArrowTip PolyLin...
5,1,TOOL,"{ActionTool, DragTracker, SelectAreaTracker, F...",TOOL,ActionTool DragTracker SelectAreaTracker Follo...
4,1,FIGURE_DECORATOR,"{BouncingDrawing, AnimationDecorator, Standard...",FIGURE_DECORATOR,BouncingDrawing AnimationDecorator StandardDra...
3,1,COMMAND_CLIPBOARD,"{CutCommand, DeleteCommand, FigureTransferComm...",COMMAND_CLIPBOARD,CutCommand DeleteCommand FigureTransferCommand...
2,1,COMMAND,"{GroupCommand, ChangeAttributeCommand, AlignCo...",COMMAND,GroupCommand ChangeAttributeCommand AlignComma...


In [15]:

o_nodes = pd.read_csv("jhotdrawcompacted-0-nodes.csv")
o_edges = pd.read_csv("jhotdrawcompacted-0-edges.csv")

remains = o_nodes[~o_nodes['name'].isin(all)].copy()
remains.drop(columns=['type'], inplace=True)
remains['level'] = [0 for _ in remains['name']]
remains['id'] = remains['name']
remains['members'] = [set([name]) for name in remains['name']]
remains['members_str'] = remains['name']

remains

,id,name,level,members,members_str
24,Figure,Figure,0,{Figure},Figure
49,ColorEntry,ColorEntry,0,{ColorEntry},ColorEntry
58,PaletteLayout,PaletteLayout,0,{PaletteLayout},PaletteLayout


In [16]:
connections = []

for _, x in c_nodes.iterrows():
  for _, y in c_nodes.iterrows():
    x_nodes = list(o_nodes.loc[o_nodes['name'].isin(x['members'])]['id'])
    y_nodes = list(o_nodes.loc[o_nodes['name'].isin(y['members'])]['id'])
    x2y = list(o_edges.loc[o_edges['source'].isin(x_nodes) & o_edges['target'].isin(y_nodes)]['interaction'])
    for interaction in x2y:
      connections.append({
        'source': x['name'],
        'target': y['name'],
        'interaction': interaction,
      })

connections

c_edges = pd.DataFrame(connections)

df = c_edges
pd.concat([df.head(5), df.tail(5)])

,source,target,interaction
0,FIGURE_ATTRIBUTE,FIGURE_ATTRIBUTE,constructs
1,FIGURE_ATTRIBUTE,FIGURE_ATTRIBUTE,depends
2,FIGURE_ATTRIBUTE,FIGURE_ATTRIBUTE,depends
3,FIGURE_ATTRIBUTE,FIGURE_ATTRIBUTE,specializes
4,FIGURE_ATTRIBUTE,FIGURE_ATTRIBUTE,constructs
1149,COMMAND,COMMAND,specializes
1150,COMMAND,COMMAND,constructs
1151,COMMAND,COMMAND,specializes
1152,COMMAND,COMMAND,constructs
1153,COMMAND,COMMAND,specializes


In [17]:
import requests

exec(open("p4c_init.py").read())
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

Loading Javascript client ... bd826595-dcaf-4e5f-9a21-11c612f55698 on https://jupyter-bridge.cytoscape.org


<IPython.core.display.Javascript object>

In [18]:
import py4cytoscape as p4c

p4c.cytoscape_ping()
p4c.cytoscape_version_info()

You are connected to Cytoscape!


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.9.1',
 'automationAPIVersion': '1.6.0',
 'py4cytoscapeVersion': '1.5.0'}

In [19]:
p4c.set_visual_style('Marquee')
p4c.create_network_from_data_frames(c_nodes.drop(columns=['members']), c_edges, title="JHotDraw-highlevel-louvain", collection="SoftwareKnowledge")

Applying default style...
Applying preferred layout


81307